In [1]:
import cv2
from mtcnn import MTCNN
import numpy as np

# Load the MTCNN face detector
detector = MTCNN()

# Load the pre-trained gender classification model
model_path = r"E:\ML Practice\Testmodel\gender_net.caffemodel"
weights_path = r"E:\ML Practice\Testmodel\gender_deploy.prototxt"
net = cv2.dnn.readNetFromCaffe(weights_path, model_path)

# Load the labels for gender classification
gender_list = ['Male', 'Female']

# Open a connection to the CCTV camera
cap = cv2.VideoCapture(0)  # Replace with your CCTV feed URL or video file path

# Define the threshold for anomaly detection
threshold = 2  # Example threshold: More than 2 men surrounding a woman

def send_alert(message):
    # Implement your alert mechanism here (e.g., email, SMS, etc.)
    print(f"Alert: {message}")

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    # Check if frame is successfully captured
    if not ret:
        print("Error: Unable to capture video.")
        break

    # Detect faces in the frame using MTCNN
    faces = detector.detect_faces(frame)
    
    # Initialize counters for each frame
    male_count = 0
    female_count = 0
    women_surrounded_by_men = []

    # Process each detected face
    for face in faces:
        x, y, width, height = face['box']
        face_region = frame[y:y+height, x:x+width]
        
        # Prepare the face for the gender classification model
        blob = cv2.dnn.blobFromImage(face_region, 1.0, (227, 227), (78.4263377603, 87.7689143744, 114.895847746), swapRB=False)
        net.setInput(blob)
        gender_preds = net.forward()
        gender = gender_list[gender_preds[0].argmax()]
        
        # Update the counters
        if gender == 'Male':
            male_count += 1
        else:
            female_count += 1

        # Draw a rectangle around the face and put text
        cv2.rectangle(frame, (x, y), (x+width, y+height), (0, 255, 0), 2)
        cv2.putText(frame, f"{gender}", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # If the face is a woman, check surrounding males
        if gender == 'Female':
            surrounding_males = 0
            for other_face in faces:
                if other_face != face:
                    ox, oy, o_width, o_height = other_face['box']
                    # Check if the other face is close enough to be considered "surrounding"
                    if (ox < x + width and ox + o_width > x) and (oy < y + height and oy + o_height > y):
                        surrounding_males += 1
            
            # If the number of surrounding males exceeds the threshold, record the woman
            if surrounding_males > threshold:
                women_surrounded_by_men.append((x, y, width, height))
    
    # Generate messages based on the detected scenario
    if female_count > 0 and male_count > female_count:
        if women_surrounded_by_men:
            send_alert(f"{len(women_surrounded_by_men)} woman/women surrounded by more than {threshold} men!")
        else:
            send_alert(f"Woman detected! Number of men ({male_count}) is more than women ({female_count}).")

    # Display the frame
    cv2.imshow('Gender Classification', frame)
    
    # Print counts
    print(f"Male: {male_count}, Female: {female_count}")

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
Male: 0, Female: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
Male: 1, Female: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━